In [42]:
import time
from ore_algebra import *
DOP, t, D = DifferentialOperators()
# The following may be overridden in input file, but need not be specified there
#
#
defaultPrecision=1e-90;
# Do not forget to increase the precision of initial conditions if going beyond e-100
#
#
#
precision=[]; #if empty we will use the default precision for all systems
periodsOnly=true; 
paths=[]; #unless overridden we will convert this to a list of straight paths

#load("~/git-projects/periods/integrate/data/current.sage")
load("~/git-projects/period-suite/current.sage")
steps=len(allODEs);
cohomologies=[];
allTransMats=[1..steps];

if len(paths)==0:
    paths=[[0,1] for j in [1..steps]]
    
if len(precision)==0:
    precision=[defaultPrecision for j in [1..steps]]


t0=time.time();


print "Integrating systems", 1, "through", steps;
for j in [1..steps]:
    if periodsOnly and j==steps:
        noOfdeqs=geometricGenus;
    else:
        noOfdeqs=len(allODEs[j-1]);
    deqs=[deq.numerator() for deq in allODEs[j-1][0:noOfdeqs]];
    print "";
    print "Integrating system number", j;
    print "";
    %time transitionMatricies = [deqs[i-1].numerical_transition_matrix(paths[j-1], precision[j-1],assume_analytic=true) for i in [1..noOfdeqs]]
    allTransMats[j-1]=transitionMatricies;
    print "Maximal error in transition matricies:"
    print [max(tm.apply_map(lambda x : x.diameter()).list()) for tm in transitionMatricies]
    cohomology=[1..noOfdeqs];
    for i in [1..noOfdeqs]:
        if j==1:
            init=Matrix(inits[i-1]);
            a = init.nrows(); b = init.ncols();
            init =  MatrixSpace(ComplexBallField(300), a, b)(init);
        else:
            init=Matrix(rawInits[j-2][i-1])*cohomologies[j-2];
        transitionMatrix=transitionMatricies[i-1];
        cohomology[i-1]=(transitionMatrix * init).row(0);
    cohomologies=cohomologies+[Matrix(cohomology)];
    
periodsWithError=Matrix(cohomologies[-1].rows()[0:geometricGenus]);
maximalError=max(periodsWithError.apply_map(lambda x : x.diameter()).list());
periods=periodsWithError.apply_map(lambda x : x.mid());
print "\nAccumulated maximal error:", maximalError
t1=time.time();
print "\nIt took", t1-t0, "seconds in total.\n"
print periods


Integrating systems 1 through 5

Integrating system number 1

CPU times: user 12.5 s, sys: 179 ms, total: 12.7 s
Wall time: 12.7 s
Maximal error in transition matricies:
[1.0962071e-90, 1.0962071e-90, 2.2809032e-91, 2.2809032e-91, 2.5646227e-90, 3.9936370e-91, 2.5646227e-90, 2.2809032e-91, 2.2809032e-91, 2.5646227e-90, 3.9936370e-91, 2.5646227e-90, 2.0153959e-90, 7.0225916e-91, 2.5646227e-90, 3.9936370e-91, 2.5646227e-90, 2.0153959e-90, 7.0225916e-91, 2.9675686e-89, 2.9675686e-89]

Integrating system number 2

CPU times: user 8.82 s, sys: 148 ms, total: 8.97 s
Wall time: 9.1 s
Maximal error in transition matricies:
[1.0653794e-94, 1.0653794e-94, 8.4610940e-93, 1.3668819e-92, 2.2231412e-97, 5.1235589e-95, 1.3484614e-97, 8.4610940e-93, 1.3668819e-92, 2.2231412e-97, 5.1235589e-95, 1.3484614e-97, 1.5014477e-96, 1.2857772e-95, 2.2231412e-97, 5.1235589e-95, 1.3484614e-97, 1.5014477e-96, 1.2857772e-95, 2.8636684e-92, 2.8636684e-92]

Integrating system number 3

CPU times: user 29.7 s, sys: 40

In [43]:
# Only for K3s
##
## Try a few values for "scale", do not exceed the negative exponent of accumulated error
##
scale = 40
##
## verify that you found honest relations by checking suspected relations using more digits
##
verify= 60
##
##

periodVector=periods
u1 = periodVector.apply_map(real)[0].list()
u2 = periodVector.apply_map(imag)[0].list()
M = matrix([u1,u2])

proj = M.transpose()
Proj = (10^scale*proj).apply_map(lambda x : 10^10*x.round())
lattice = block_matrix([[Proj, matrix.identity(Proj.dimensions()[0])]])
reducedLattice = lattice.LLL()

Proj2 = (10^verify*proj).apply_map(lambda x : x.round())
testRelations = [v[2:23] * Proj2 for v in reducedLattice.rows()]

norms = [testRelations[i].norm().n()+reducedLattice.row(i)[2:23].norm().n() for i in [0..20]]


sortedNorms=sorted(norms)
consecutiveRatios = [sortedNorms[i]/sortedNorms[i-1] for i in [1..20]]
j=0; rank=1;
while j <= 19 :
    if consecutiveRatios[j] > 100 :
        j=20
    rank = rank + 1
    j=j+1
    
if norms[0] > 50 :
    rank = 1

print "\nPicard rank appears to be", rank, "\n"
print "Check to see if this makes sense:\n"

print "01","||", "---hyperplane section---"
j=0
while j <= 7 :
    print "0" + str(j+2), "||", norms[j], testRelations[j]
    j=j+1
while j <= 20 :
    print j+2, "||", norms[j], testRelations[j]
    j=j+1


Picard rank appears to be 18 

Check to see if this makes sense:

01 || ---hyperplane section---
02 || 1.41421356237310 (0, 0)
03 || 1.41421356237310 (0, 0)
04 || 1.41421356237310 (0, 0)
05 || 1.41421356237310 (0, 0)
06 || 1.41421356237310 (0, 0)
07 || 1.41421356237310 (0, 0)
08 || 1.41421356237310 (0, 0)
09 || 1.41421356237310 (0, 0)
10 || 1.41421356237310 (0, 0)
11 || 1.41421356237310 (0, 0)
12 || 1.41421356237310 (0, 0)
13 || 1.41421356237310 (0, 0)
14 || 1.41421356237310 (0, 0)
15 || 1.41421356237310 (0, 0)
16 || 1.41421356237310 (0, 0)
17 || 1.41421356237310 (0, 0)
18 || 1.41421356237310 (0, 0)
19 || 3.33905142651253e44 (-286379264940093624022119131627068382463861793, 171696129546723892260571598536465249001151910)
20 || 5.98264703715313e44 (-545938922617635990913403542181141926388480059, 244686224546184802443832004988592586417797117)
21 || 8.16902434229263e44 (757160608615082683671505521395104133146474897, -306655180636709270184063405742342727443148027)
22 || 1.25383049289246e44 

In [47]:
reducedLattice.row(16)

(0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1)